In [11]:
import pandas as pd
import re
import os
import random



dirname = os.path.abspath('C:/Users/lucas/workspace/dashtutorial')
arquivo_in=dirname + '/data/git_changes.csv'
arquivo_date_in=dirname + '/data/git_by_date.csv'
arquivo_stats_in=dirname + '/data/git_stats.txt'
arquivo_git_name_only=dirname + '/data/git_name_only.txt'
arquivo_out=dirname + '/data/git_changes_norm.csv'

def extract_hash(data):
    match = re.search(r'(^[\S]{9})', data)
    if data.startswith(" "):
        return pd.NA
    else:
        try:
            return match.group(1)
        except AttributeError:
            return pd.NA

def extract_hash_files(data):
    prefixes = ['A', 'M', 'D', 'R']
    if any(map(data.startswith, prefixes)):
        return pd.NA
    else:
        try:
            match = re.search(r'(^[\S]{9})', data)
            return match.group(1)
        except AttributeError:
            return pd.NA

df_changes=pd.read_csv(arquivo_in,names=['data'],sep="|",encoding="utf8", error_bad_lines=False)

colnames_date=['hash','user','date'] 
df_date=pd.read_csv(arquivo_date_in,names=colnames_date,sep=';', error_bad_lines=False)

df_changes['hash'] = df_changes['data'].apply(extract_hash)
df_changes['hash'].ffill(inplace=True)

df_changes['linsertion'] = df_changes['data'].str.extract('([\d]{1,} insertion)')
df_changes['ldeletion'] = df_changes['data'].str.extract('([\d]{1,} file)')
df_changes['fchanged'] = df_changes['data'].str.extract('([\d]{1,} deletion)')

df_changes['linsertion'] = df_changes['linsertion'].str.extract('([\d]{1,})')
df_changes['ldeletion'] = df_changes['ldeletion'].str.extract('([\d]{1,})')
df_changes['fchanged'] = df_changes['fchanged'].str.extract('([\d]{1,})')

df_changes['linsertion'] = pd.to_numeric(df_changes['linsertion'])
df_changes['ldeletion'] = pd.to_numeric(df_changes['ldeletion'])
df_changes['fchanged'] = pd.to_numeric(df_changes['fchanged'])

df_changes.fillna(0,inplace=True)

df_changes['ldiff'] = df_changes['linsertion'] - df_changes['ldeletion']

df_changes_minor = df_changes.loc[df_changes['data'].str.contains("#minor")]
df_changes = df_changes.loc[df_changes['data'].str.startswith(" ")]

df_changes.set_index("hash", inplace=True)
df_changes_minor.set_index("hash", inplace=True)
df_date.set_index("hash", inplace=True)
df_date['date'] = pd.to_datetime(df_date['date'], format=' %a %b %d %H:%M:%S %Y')
df_date['monthdate'] = df_date['date'].dt.to_period('M')

df_full = df_changes.join(df_date)
df_full['commit_category'] = pd.Series(random.choices(['FEATURE','BUG','HOTFIX'], weights=[1, 1, 1], k=len(df_full)),index=df_full.index)

df_full

df_git_files = pd.read_csv(arquivo_git_name_only,names=['data'], error_bad_lines=False)
df_git_files['hash'] = df_git_files['data'].apply(extract_hash_files)
df_git_files['hash'].ffill(inplace=True)
df_git_files = df_git_files.loc[df_git_files['data'].str.contains('/t')]
print(df_git_files)
#print(df_git_files.data.str.split('\t',expand=True))
df_git_files[['tipo','arquivo','colc']] = df_git_files.data.str.split('\t',expand=True)
df_git_files[['arquivo','extension']] = df_git_files.loc[df_git_files['arquivo'].str.contains('\\.')].arquivo.str.rsplit(".",n=1,expand=True)

Empty DataFrame
Columns: [data, hash]
Index: []


ValueError: Columns must be same length as key

In [29]:
def extract_hash_files(data):
    prefixes = ['A', 'M', 'D', 'R']
    if any(map(data.startswith, prefixes)):
        return pd.NA
    else:
        try:
            match = re.search(r'(^[\S]{5,9})', data)
            return match.group(1)
        except AttributeError:
            print('error')
            return pd.NA

df_git_files = pd.read_csv(arquivo_git_name_only,names=['data'], error_bad_lines=False)
df_git_files['hash'] = df_git_files['data'].apply(extract_hash_files)
df_git_files[['tipo','arquivo']] = df_git_files.data.str.split('\t',expand=True)
df_git_files[['arquivo','extension']] = df_git_files.loc[df_git_files['arquivo'].str.contains('\\.')].arquivo.str.rsplit(".",n=1,expand=True)
df_git_files

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [38]:
text = '1dde1e7 First Commit	'
prefixes = ['A', 'M', 'D', 'R']
#if any(map(text.startswith, prefixes)):
#    print(text)
#else:
#    print(text)
#    match = re.search(r'(^[\S]{9})', text)
#    match.group(1)    

match = re.search(r'(^[\S]{7})', text)
df_git_files[['tipo','arquivo']] = df_git_files.data.str.split('\t',expand=True)

df_git_files[['arquivo','extension']] = df_git_files.loc[~pd.isnull(df_git_files['arquivo'])].arquivo.str.rsplit(".",n=1,expand=True)
df_git_files.loc[~pd.isnull(df_git_files['arquivo'])]
print(match)

AttributeError: 'DataFrame' object has no attribute 'str'